In [ ]:
from flask import Flask, render_template, request
import os
import pickle
import numpy as np
from scipy.stats import poisson,skellam
import pandas as pd
from datetime import date
from update import update_model
from apiRequest import getFutureFixture

In [ ]:
predictor = pickle.load(open(os.path.join('predictor/plk_objects/predictor.plk'), 'rb'))

In [ ]:
predictor

In [ ]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

In [ ]:
future_fixtures = getFutureFixture()

In [ ]:
future_fixtures

In [ ]:
def changeClubName(matches):

    for match in matches:
        home = match[1]
        away = match[2]

        if match[1] == 'Manchester City':
            match[1] = 'Man City'
        elif match[1] == 'Manchester United':
            match[1] = 'Man United'
        elif match[1] == 'Sheffield Utd':
            match[1] = 'Sheffield United'

        if match[2] == 'Manchester City':
            match[2] = 'Man City'
        elif match[2] == 'Manchester United':
            match[2] = 'Man United'
        elif match[2] == 'Sheffield Utd':
            match[2] = 'Sheffield United'
            
    return matches

upcoming_fixtures = changeClubName(future_fixtures)

In [ ]:
upcoming_fixtures

In [ ]:
home_prop = []
draw_prop = []
away_prop = []

for index in upcoming_fixtures:
    home = index[1]
    away = index[2]
    result = simulate_match(predictor, home, away, max_goals=10)
    home_prop.append(np.sum(np.tril(result, -1)))
    draw_prop.append(np.sum(np.diag(result)))
    away_prop.append(np.sum(np.triu(result, 1)))

In [ ]:
print(predictor)

In [ ]:
result = simulate_match(predictor, home, away, max_goals=10)

In [ ]:
result